## Don't Run All Make sure to convert to live later

In [1]:
from datetime import datetime
from fund.fund import Fund
from pricer.pricer import Pricer as pricer_list
from ranker.ranker import Ranker as ranker_list
from classifier.classifier import Classifier as classifier_list
from portfolio.aportfolio import APortfolio
from parameters.parameters import Parameters 
import pandas as pd
from returns.products import Products
from processor.processor import Processor as p
from alpaca_api.alpaca_api import AlpacaApi

In [2]:
alp = AlpacaApi()

In [3]:
portfolio = APortfolio(pricer_list.WEEKLY_STOCK_SPECULATION
                          ,classifier_list.WEEKLY_STOCK_SPECULATION_CLASSIFIER
                          ,ranker_list.WEEKLY_STOCK_ROLLING_RANKER)
portfolios = [portfolio]

In [4]:
backtest_start_date = datetime(2023,1,1)
backtest_end_date  = datetime.now()
current_start_date = datetime.now()

In [5]:
fund = Fund(portfolios,backtest_start_date,backtest_end_date,current_start_date)

In [6]:
fund.initialize_portfolios()
fund.initialize_backtesters()

In [7]:
test_portfolio = fund.portfolios[0]
risk_returns = test_portfolio.create_risk_returns()
sim = test_portfolio.create_current_simulation()
merged = test_portfolio.merge_sim_returns(sim,risk_returns)

In [8]:
merged

,year,week,ticker,price_prediction,rank_prediction,classification_prediction,close,high,low,open,...,quarter,risk_return_2,risk_return_3,risk_return_4,day,weekly_risk_return,bench_weekly_return,weekly_variance,market_weekly_cov,weekly_beta
0,2023,2,MMM,122.552902,120.771538,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023,2,AOS,58.700774,55.362971,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023,2,ABT,108.663675,102.858404,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023,2,ABBV,160.204013,145.692922,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023,2,ABMD,381.647428,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11216,2023,24,YUM,132.211526,131.573586,1.0,134.17,135.19,133.75,134.540,...,2.0,0.014518,0.042583,0.049926,0.0,0.049926,0.014693,0.000094,0.000111,1.175260
11217,2023,24,ZBRA,272.210026,294.525350,1.0,269.11,276.44,267.53,276.350,...,2.0,0.010932,0.024908,0.004967,0.0,0.004967,0.014693,0.000094,0.000167,1.768091
11218,2023,24,ZBH,132.211526,129.178231,1.0,130.47,130.70,129.62,130.000,...,2.0,0.024419,0.024580,0.031954,0.0,0.031954,0.014693,0.000094,-0.000006,-0.058300
11219,2023,24,ZION,30.965639,36.999472,1.0,28.72,29.65,28.60,29.285,...,2.0,0.037947,0.052400,-0.006572,0.0,-0.006572,0.014693,0.000094,0.000371,3.927887


In [9]:
parameter = {
    "value":True,
    "ceiling":True,
    "classification":False,
    "rank":False,
    "short":False,
    "market_return":1.5
}

In [10]:
stuff = test_portfolio.recommendation(merged,parameter)

# SET THE CURRENT WEEK

In [11]:
set_the_current_Week

NameError: name 'set_the_current_Week' is not defined

In [12]:
rec = stuff[0]
week = 24
trades = rec[rec["week"]==week].head(10)
trades

,year,week,ticker,weekly_delta,weekly_delta_sign,position,value,ceiling,classification,rank,short,market_return
18,2023,24,NVR,0.196458,1.0,0,True,True,False,False,False,1.5
37,2023,24,CPB,0.055269,1.0,1,True,True,False,False,False,1.5
56,2023,24,AMCR,0.046057,1.0,2,True,True,False,False,False,1.5
75,2023,24,DG,0.039470,1.0,3,True,True,False,False,False,1.5
94,2023,24,CHD,0.038046,1.0,4,True,True,False,False,False,1.5
113,2023,24,AVGO,0.036791,1.0,5,True,True,False,False,False,1.5
132,2023,24,CLX,0.030981,1.0,6,True,True,False,False,False,1.5
151,2023,24,MCD,0.030003,1.0,7,True,True,False,False,False,1.5
170,2023,24,BIO,0.029703,1.0,8,True,True,False,False,False,1.5
189,2023,24,ILMN,0.026486,1.0,9,True,True,False,False,False,1.5


In [16]:
account = alp.paper_get_account()
cash = float(account.cash) - 10

In [17]:
fail

NameError: name 'fail' is not defined

In [18]:
order_data = []
for row in trades.iterrows():
    ticker = row[1]["ticker"]
    amount = round(cash / 10) 
    print(ticker,amount)
#     order_data.append(alp.paper_market_order(ticker,amount))
    

NVR 9999
CPB 9999
AMCR 9999
DG 9999
CHD 9999
AVGO 9999
CLX 9999
MCD 9999
BIO 9999
ILMN 9999


In [ ]:
## close all
# close_data = alp.paper_close_all()